In [1]:
# Character level tokenisation

# Step 1: Input text
text = "hello world"

# Step 2: Get all unique characters in the text
vocab = sorted(list(set(text)))
print("Vocabulary:", vocab)

# Step 3: Create mappings: char → int and int → char
char_to_int = {ch: i for i, ch in enumerate(vocab)}
int_to_char = {i: ch for i, ch in enumerate(vocab)}

print("\nCharacter to Integer mapping:")
print(char_to_int)

# Step 4: Encode (text → integers)
encoded = [char_to_int[ch] for ch in text]
print("\nEncoded sequence:", encoded)

# Step 5: Decode (integers → text)
decoded = ''.join([int_to_char[i] for i in encoded])
print("\nDecoded text:", decoded)


Vocabulary: [' ', 'd', 'e', 'h', 'l', 'o', 'r', 'w']

Character to Integer mapping:
{' ': 0, 'd': 1, 'e': 2, 'h': 3, 'l': 4, 'o': 5, 'r': 6, 'w': 7}

Encoded sequence: [3, 2, 4, 4, 5, 0, 7, 5, 6, 4, 1]

Decoded text: hello world


In [2]:
# Bigram Model

# Count bigrams (pairs of consecutive chars)
import torch
vocab_size=len(vocab)
# Create a count matrix (vocab_size x vocab_size)
N = torch.zeros((vocab_size, vocab_size), dtype=torch.int32)
for ch1, ch2 in zip(encoded, encoded[1:]):
    N[ch1, ch2] += 1
print("Bigram count matrix:\n", N)


# Convert counts → probabilities
# Add 1 for smoothing (avoid division by zero)
P = (N + 1).float()
P /= P.sum(1, keepdim=True)
print("\nProbability matrix:\n", P)


# Generate text using the bigram model
import torch
# Start with a random character
ix = torch.randint(0, vocab_size, (1,)).item()
out = [ix]
for _ in range(20):  # generate 20 characters
    # Sample next char based on probability
    probs = P[ix]
    ix = torch.multinomial(probs, num_samples=1).item()
    out.append(ix)
# Decode indices to characters
generated_text = ''.join([int_to_char[i] for i in out])
print("\nGenerated text:", generated_text)

/Users/pornimagaikwad/GPT_from_Scratch/venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Bigram count matrix:
 tensor([[0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0]], dtype=torch.int32)

Probability matrix:
 tensor([[0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.2222],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.2222, 0.1111, 0.1111, 0.1111],
        [0.1111, 0.1111, 0.2222, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111],
        [0.0909, 0.1818, 0.0909, 0.0909, 0.1818, 0.1818, 0.0909, 0.0909],
        [0.2000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.2000, 0.1000],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.2222, 0.1111, 0.1111, 0.1111],
        [0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.2222, 0.1111, 0.1111]])

Generated text: rldeldlldlwohhr r lwe
